# Multi-dimensional arrays and **Tensor** class
### Last modification (05.04.2018).

In this tutorial we will show the core data structures of multidimenaional arrays within tensor algebra and illustrate how they are integrated into [hottbox](https://github.com/hottbox/hottbox). For more details visit the [documentation page](https://hottbox.github.io/stable/api/hottbox.core.html#module-hottbox.core).

**Author:** Ilya Kisil - ilyakisil@gmail.com

In [1]:
import numpy as np
from hottbox.core.structures import Tensor

<center><img src="./images/tensors.png" width="850" align="center"/></center>

Tensor is a multi-dimenaional array of data where each dimension is conventionally referred to as **mode**. Its order is defined by the number of its modes which is equivivalent to the number of indices required to identify a particular entry of a multi-dimensional array.
For example, an element of a third order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ can be written in general form as:

$$ x_{ijk} = \mathbf{\underline{X}}[i, j, k]$$

In order to create tensor using ``hottbox``, you simply need to pass numpy ndarray to the constructor of the ``Tensor`` class. 

> **Note:** the ND-array is stored in the attribute `data`.

This will allow you to use top level API for the most common properties and operations on the tensor itself that correspond to the conventional definitions. 

> **Note:** In order to be consistent with python indexing, count of modes starts from zeros.

In [2]:
array_3d = np.arange(24).reshape((2, 3, 4))
tensor = Tensor(array_3d)
tensor.data

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

As mentioned previously, the conventional names of the tensor characteristics (e.g. order, shape, size) are preserved for the objects of **`Tensor`** class.

In [3]:
print('This tensor is of order {}.'.format(tensor.order))
print('The sizes of its modes are {} respectively.'.format(tensor.shape))
print('It consists of {} elemetns.'.format(tensor.size))
print('Its Frobenious norm = {:.2f}'.format(tensor.frob_norm))

This tensor is of order 3.
The sizes of its modes are (2, 3, 4) respectively.
It consists of 24 elemetns.
Its Frobenious norm = 65.76


# Fundamental operations with the obejcts of **Tensor** class

Next, let's have a look at the fundamental operation with a tensor and how to apply them to the object of class **``Tensor``**. We shall start from defining the main substructures of a tensor. 
For ease of visualisation and compact notation, we consider a third order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$.

<center><img src="./images/tensor_substructures.png" width="600" align="center"/></center>

1. A **fiber** is a vector obtained by fixing all but one of the indices, e.g.  $\mathbf{\underline{X}}[i,:,k]$ is the mode-2 fiber (usually refered to as row fiber). 

- Fixing all but two of the indices yields a matrix called a **slice** of a tensor, e.g. $\mathbf{\underline{X}}[:,:,k]$ is the mode-[1,2] slice (usually refered to as frontal slice).

> **Note:** The same principals and definitions can be applied to a tensor of arbitrarily large order. On top of that, one can obtain a **subtensor** by fixing at least three indecies and let other vary.

### **Unfolding a tensor**

Conventionally, unfolding is considered to be a process of element mapping from a tensor to a matrix. In other words, it arranges the mode-$n$ fibers of a tensor to be the columns of the matrix and denoted as:

$$\mathbf{\underline{A}} \xrightarrow{n} \mathbf{A}_{(n)}$$

Thus, this operations requires to specify a mode along which a tensor will be unfolded. For a third order tensor, a visually representation of such operation is as following:

<center><img src="./images/unfolding.png" width="600" align="center"/></center>

> **Note:** it can be extended to a more general case, when one converts a tensor of order $N$ into a tensor of order $M$ where $N > M$. In this case, one would need to specify a set of modes along which a tensor will be unfolded. 

In **``hottbox``** this functionality is available through the corresponding methods of the **``Tensor``** class:

```python
tensor.unfold(mode=0)
```

By default, it changes the state of a tensor. If you want to get unfolded tensor as a new object then use the following:

```python
tensor_unfolded = tensor.unfold(mode=0, inplace=False)
```

In [4]:
array_3d = np.arange(24).reshape((2, 3, 4))
tensor = Tensor(array_3d)
tensor.data

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [5]:
tensor_unfolded = tensor.unfold(mode=0, inplace=False)
tensor_unfolded.data

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]])

In [6]:
tensor_unfolded = tensor.unfold(mode=1, inplace=False)
tensor_unfolded.data

array([[ 0,  1,  2,  3, 12, 13, 14, 15],
       [ 4,  5,  6,  7, 16, 17, 18, 19],
       [ 8,  9, 10, 11, 20, 21, 22, 23]])

In [7]:
tensor_unfolded = tensor.unfold(mode=2, inplace=False)
tensor_unfolded.data

array([[ 0,  4,  8, 12, 16, 20],
       [ 1,  5,  9, 13, 17, 21],
       [ 2,  6, 10, 14, 18, 22],
       [ 3,  7, 11, 15, 19, 23]])

In [8]:
tensor.unfold(mode=2)
tensor.data

array([[ 0,  4,  8, 12, 16, 20],
       [ 1,  5,  9, 13, 17, 21],
       [ 2,  6, 10, 14, 18, 22],
       [ 3,  7, 11, 15, 19, 23]])

In [9]:
tensor_unfolded is tensor

False

### **Folding of a tensor**

Folding is most commonly referred to a process of element mapping from a matrix or a vector to a tensor. However, it can be extended to a more general case, when one converts a tensor of order $N$ into a tensor of order $M$ where $N < M$.

<center><img src="./images/folding.png" width="500" align="center"/></center>

In **``hottbox``** this functionality is available through the corresponding methods of the **``Tensor``** class:

```python
tensor_unfolded.fold()
```

By default, it changes the state of a tensor. If you want to get folded tensor as a new object then use the following:

```python
tensor_folded = tensor_unfolded.fold(inplace=False)
```

> **Note:** in **``hottbox``** v0.1.1 this functionality merely reverts the unfolding operation.

In [10]:
array_3d = np.arange(24).reshape((2, 3, 4))
tensor = Tensor(array_3d)
tensor.data

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [11]:
tensor_unfolded = tensor.unfold(mode=1, inplace=False)
tensor_unfolded.data

array([[ 0,  1,  2,  3, 12, 13, 14, 15],
       [ 4,  5,  6,  7, 16, 17, 18, 19],
       [ 8,  9, 10, 11, 20, 21, 22, 23]])

In [12]:
tensor_folded = tensor_unfolded.fold(inplace=False)
tensor_folded.data

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [13]:
tensor.unfold(mode=1)
tensor.data

array([[ 0,  1,  2,  3, 12, 13, 14, 15],
       [ 4,  5,  6,  7, 16, 17, 18, 19],
       [ 8,  9, 10, 11, 20, 21, 22, 23]])

In [14]:
tensor.fold()
tensor.data

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

### **Mode-$n$ product**

The mode-$n$ product is the multiplication of a tensor  by a matrix along the $n^{th}$ mode of a tensor. This essentially means that each mode-$n$ fiber should be multiplied by this matrix. Mathematically, this is expressed as:

$$\mathbf{\underline{X}} \times_n \mathbf{A} = \mathbf{\underline{Y}} \quad \Leftrightarrow  \quad \mathbf{Y}_{(n)} = \mathbf{A} \mathbf{X}_{(n)}  $$

<center><img src="./images/mode_n_product.png" width="500" align="center"/></center>

Important properties of the mode-$n$ product:

1. For distinct modes in a series of multiplications, the order of the multiplication is irrelevent: 

    $$\mathbf{\underline{X}} \times_n \mathbf{A} \times_m \mathbf{B} = \mathbf{\underline{X}} \times_m \mathbf{B} \times_n \mathbf{A} \quad (m \neq n)$$

- However, it does not hold if the modes are the same :

    $$\mathbf{\underline{X}} \times_n \mathbf{A} \times_n \mathbf{B} = \mathbf{\underline{X}} \times_n (\mathbf{B}\mathbf{A})$$

In **``hottbox``** mode-$n$ product is available through the corresponding method of the **``Tensor``** class:

```python
tensor.mode_n_product(matrix, mode=n)
```

By default, it changes the state of a tensor. If you want to get an unfolded copy of a tensor then use the following:

```python
tensor.mode_n_product(matrix, mode=n, inplace=False)
```

In the following example, we will consider the sequence of mode-$n$ products:

$$\mathbf{\underline{X}} \times_2 \mathbf{A} \times_3 \mathbf{B} = \mathbf{\underline{Y}}$$
$$\mathbf{\underline{X}} \times_3 \mathbf{B} \times_2 \mathbf{A} = \mathbf{\underline{Z}}$$

Where $\mathbf{\underline{X}} \in \mathbb{R}^{2 \times 3 \times 4}, \mathbf{A} \in \mathbb{R}^{5 \times 3}$ and $\mathbf{B} \in \mathbb{R}^{6 \times 4}$. Thus, the resulting tensors $\mathbf{\underline{Y}}, \mathbf{\underline{Z}}$ will be equal and of shape (2,5,6), e.g. $\mathbf{\underline{Y}} \in \mathbb{R}^{2 \times 6 \times 5}$

> **Note:** in **``hottbox``** v0.1.1 a **``matrix``** should be a numpy array.

In [15]:
I = 2
J = 3
K = 4

J_new = 5
K_new = 6

array_3d = np.arange(I * J * K).reshape(I, J ,K)
X = Tensor(array_3d)

A = np.arange(J_new * J).reshape(J_new, J)
B = np.arange(K_new * K).reshape(K_new, K)

print('The initial shape of tensor X is {}'.format(X.shape))

The initial shape of tensor X is (2, 3, 4)


In order to perform a sequence of mode-$n$ products, simply chain methods. 

In [16]:
Y = X.mode_n_product(A, mode=1, inplace=False).mode_n_product(B, mode=2, inplace=False)

# Perform mode-n product in reversed order
Z = X.mode_n_product(B, mode=2, inplace=False).mode_n_product(A, mode=1, inplace=False)

Here, despite the **`Y`** and **`Z`** are being different objects, their attribute **`data`** will have the same values.

In [17]:
print('The shape of tensor Y is {}'.format(Y.shape))
print('The shape of tensor Z is {}'.format(Z.shape))
print('The underlying data arrays are equal for both of them:')
Y.data - Z.data

The shape of tensor Y is (2, 5, 6)
The shape of tensor Z is (2, 5, 6)
The underlying data arrays are equal for both of them:


array([[[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]])

Here we will change the state of the original tensor, instead of creating new objects of **`Tensor`** class.

In [18]:
X.mode_n_product(A, mode=1).mode_n_product(B, mode=2)
print('The shape of tensor X is {}'.format(X.shape))
print('The underlying data array is equal to the previously obtained:')
X.data - Y.data

The shape of tensor X is (2, 5, 6)
The underlying data array is equal to the previously obtained:


array([[[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]])

As expected, the result remains the same.